<img style="float: right; margin: 0px 0px 15px 15px;" src="https://www.carrerasenlinea.mx/logos/original/logo-instituto-tecnologico-y-de-estudios-superiores-de-occidente.png" width="150px" height="100px" /> 


&nbsp;
# Administración activa Vs Administración pasiva del capital
## Equipo B

***Emilio de la Torre***


***José Raúl Gallardo***


***Adrian Garcia*** 


La idea es replicar un ETF con posiciones en directo. Te han asignado la tarea de hacer un ejercicio simple de replicar la construcción de un portafolio de inversión y un criterio de rebalanceo lo más parecido a un ETF de ishares que tu selecciones. Esto con la finalidad de comparar los resultados de haber rebalanceado periódicamente el portafolio, ó, no haberlo hecho y continuar todo un año con las mismas posiciones.

## Limpieza y extracción de datos

### Importando librerías a utilizar

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Importando los datos del ETF

In [42]:
dr = np.arange(9)
data_etf = pd.read_excel("IVE_holdings1.csv", sheet_name=0, header=1, index_col=None, parse_dates=False, skip_blank_lines=True, skiprows=dr)
data_etf

,Ticker,Name,Asset Class,Weight (%),Price,Shares,Market Value,Notional Value,Sector,SEDOL,ISIN,Exchange
0,XOM,EXXON MOBIL CORP,Equity,3.66,82.01,6083649.0,4.989201e+08,4.989201e+08,Energy,2326618,US30231G1022,New York Stock Exchange Inc.
1,BRKB,BERKSHIRE HATHAWAY INC CLASS B,Equity,3.40,166.68,2786587.0,4.644683e+08,4.644683e+08,Financials,2073390,US0846707026,New York Stock Exchange Inc.
2,JPM,JPMORGAN CHASE & CO,Equity,3.37,87.84,5240684.0,4.603417e+08,4.603417e+08,Financials,2190385,US46625H1005,New York Stock Exchange Inc.
3,T,AT&T INC,Equity,2.74,41.55,9010739.0,3.743962e+08,3.743962e+08,Telecommunications,2831811,US00206R1023,New York Stock Exchange Inc.
4,WFC,WELLS FARGO,Equity,2.70,55.66,6606722.0,3.677301e+08,3.677301e+08,Financials,2649100,US9497461015,New York Stock Exchange Inc.
5,BAC,BANK OF AMERICA CORP,Equity,2.54,23.59,14708577.0,3.469753e+08,3.469753e+08,Financials,2295677,US0605051046,New York Stock Exchange Inc.
6,CVX,CHEVRON CORP,Equity,2.19,107.37,2777508.0,2.982210e+08,2.982210e+08,Energy,2838555,US1667641005,New York Stock Exchange Inc.
7,CSCO,CISCO SYSTEMS INC,Equity,1.82,33.80,7347386.0,2.483416e+08,2.483416e+08,Information Technology,2198163,US17275R1023,NASDAQ
8,C,CITIGROUP INC,Equity,1.78,59.82,4065107.0,2.431747e+08,2.431747e+08,Financials,2297907,US1729674242,New York Stock Exchange Inc.
9,GE,GENERAL ELECTRIC,Equity,1.48,29.80,6784403.0,2.021752e+08,2.021752e+08,Industrials,2380498,US3696041033,New York Stock Exchange Inc.


In [ ]:
def get_data(nFiles, file_title):
    df = []
    data_etf = pd.read_excel(file_title, sheet_name=0, header=0, index_col=None, parse_dates=False, skip_blank_lines=True)

In [72]:
etf_file = 'IVEetf.xlsx' # Dirección del archivo que contiene información del etf
data_etf = pd.read_excel(etf_file, sheet_name=0, header=0, sep=',', index_col=None, parse_dates=False, skip_blank_lines=True) 
# Importando los datos desde el excel
data_etf.head()

,Ticker,Name,Asset Class,Weight (%),Price,Shares,Sector,Exchange
0,AAPL,APPLE INC,Equity,8.10013,202.64,5945684,Information Technology,NASDAQ
1,JPM,JPMORGAN CHASE & CO,Equity,3.14520,106.02,4412616,Financials,New York Stock Exchange Inc.
2,T,AT&T INC,Equity,2.32389,34.82,9927108,Communication,New York Stock Exchange Inc.
3,BAC,BANK OF AMERICA CORP,Equity,2.14052,26.47,12028190,Financials,New York Stock Exchange Inc.
4,CVX,CHEVRON CORP,Equity,2.00648,115.18,2591158,Energy,New York Stock Exchange Inc.


In [73]:
tickers = [data_etf.iloc[i,0] for i in range(len(data_etf[['Ticker']])) if data_etf.iloc[i,2] == 'Equity']
# Nos quedamos solo con los tickers que son acciones "Equity"

### Extracción de precios de Quandl

In [74]:
import quandl
quandl.ApiConfig.api_key = "JyBxQPPNx_LTemwn-VbD" # Api para acceso a Quandl

In [75]:
data = quandl.get_table('WIKI/PRICES', ticker = [tickers], qopts = {'columns': ['ticker', 'date', 'close']}, 
                        date = {'gte': '2017-03-01', 'lte': '2018-03-01'}, paginate=True)
        #De la línea anterior se obtinen los precios de las acciones en un DataFrame.
data.head()

,ticker,date,close
None,,,
0,ZTS,2018-03-01,80.44
1,ZTS,2018-02-28,80.86
2,ZTS,2018-02-27,81.67
3,ZTS,2018-02-26,83.30
4,ZTS,2018-02-23,82.79


In [76]:
#A continuación de ordena el dataframe para tener los precios de los tickers en columnas
stock_prices = data.set_index('date')
stock_prices = stock_prices.pivot(columns='ticker')
stock_prices.head()

close                                                             \
ticker          A    AAL     AAP    AAPL   ABBV    ABC     ACN    ADI    ADM   
date                                                                           
2017-03-01  52.04  47.09  156.68  139.79  62.83  92.23  124.07  83.76  46.51   
2017-03-02  51.38  45.72  157.36  138.96  63.25  91.94  124.35  83.65  45.48   
2017-03-03  51.98  46.82  155.35  139.78  63.34  92.23  123.64  83.18  45.20   
2017-03-06  51.25  45.31  154.25  139.34  63.55  89.99  123.24  83.85  44.76   
2017-03-07  51.21  44.84  151.99  139.52  63.69  88.48  124.14  83.41  44.46   

                   ...                                                       \
ticker        AEP  ...       WU     WY    WYNN     XEC    XEL    XOM   XRAY   
date               ...                                                        
2017-03-01  66.12  ...    19.91  34.07  102.71  128.45  43.50  83.02  64.50   
2017-03-02  66.50  ...    19.93  34.19  100.15  124.97  43.85  83.30  63.65   
2017-03-03  66.52  ...    19.96  34.14  101.04  125.61  43.72  82.46  63.36   
2017-03-06  66.35  ...    19.90  33.72  100.37  125.34  43.62  82.83  62.75   
2017-03-07  66.24  ...    19.90  33.47   99.46  124.25  43.82  82.52  62.55   

                                
ticker       XRX   ZION    ZTS  
date                            
2017-03-01  7.39  46.53  53.89  
2017-03-02  7.29  44.96  53.37  
2017-03-03  7.36  45.36  53.42  
2017-03-06  7.45  45.33  53.08  
2017-03-07  7.36  45.28  53.07  

[5 rows x 371 columns]

Para tener los pesos de las acciones presentes en Quandl es necesario hacer match entre los tickers del etf y los de la fuente de precios.

In [106]:
#Extraemos los tickers de Quandl.
tickers_quandl = [stock_prices.columns[i][1] for i in range(len(stock_prices.columns))]

In [107]:
df_w = pd.DataFrame(tickers_quandl, columns=['Ticker']) # Tickers se hacen un dataframe
w = df_w.set_index('Ticker').join(data_etf.set_index('Ticker'), on='Ticker') # La función join hace match entre df's.
w = w[['Weight (%)']] # Este dataframe tiene los pesos que están presentes en ambas fuentes.